In [ ]:
# Import all dependencies
import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import os
import seaborn as sns
from datetime import datetime as dt, timedelta

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc

# Conducting Analysis

In [ ]:
# Create an engine to a SQLite database file: `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite", echo = False)

# Explore Database

In [ ]:
# View table names
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# View columns and types for each table
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])
# columns

In [ ]:
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])
# columns

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys() 

In [ ]:
# Map each item
Measurements = Base.classes.measurements 
Stations = Base.classes.stations 

In [ ]:
session = Session(engine)

# Precipitation Analysis

In [ ]:
# Retrieve Latest Date to avoid the need to hard code the actual date
session.query(Measurements.date).order_by(Measurements.date.desc()).first()

In [ ]:
# Get and store date in proper format
latest_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()[0]
format_latest_date = dt.strptime(latest_date,"%Y-%m-%d")
format_latest_date 

In [ ]:
# Gather data from last 12 months
sel = [Measurements.date, Measurements.prcp]
initial_date = format_latest_date - timedelta(days=365) 
prcp_data = session.query(*sel).\
    filter((Measurements.date >= initial_date)).all()
    
prcp_data[:10] 

# Create dataframe for plot

In [ ]:
prcp_df = pd.DataFrame(prcp_data, columns=['Precipitation Date', 'Precipitation'])
prcp_df.set_index('Precipitation Date', inplace=True)

In [ ]:
prcp_df.count()

In [ ]:
prcp_df.head() 

In [ ]:
# Create precipitation analysis plot
prcp_df.plot()
plt.xlabel("Precipitation Data from 8/24/2016 through 8/23/2017")
plt.ylabel("Precipitation Amount")
plt.show()

# Summary Statistics for Precipitation Data

In [ ]:
#Use Pandas to print the summary statistics for the precipitation data.
prcp_df.describe()

In [ ]:
prcp_df.dtypes

# Station Analysis

In [ ]:
# Get station names and IDs
stations = session.query(Stations.name, Stations.station).all()
stations_df  = pd.DataFrame(stations,columns=["Station Name", "Station ID"])
stations_dict = stations_df.to_dict(orient='records')
stations_dict 

In [ ]:
# Find total number of stations
station_count = session.query(Stations.id).count()
print ("Total Number of Stations are: "+ str(station_count))

In [ ]:
# Find and display most active stations
sel = [Measurements.station, func.count(Measurements.tobs)]
active_station_data = session.query(*sel).group_by(Measurements.station).order_by(desc(func.count(Measurements.tobs))).all()
active_station_data

In [ ]:
print ("The station with the most observation data is: " + str(active_stations_data[0]))

In [ ]:
highest_station = active_station_data[0][0]

In [ ]:
# Retrieve last 12 months of temperature observation data and plot results

sel = [Measurements.tobs]
temperature_data = session.query(*sel).\
    filter(Measurements.date >= initial_date).\
    filter(Measurements.station == highest_station).all()

In [ ]:
len(temperature_data) 

In [ ]:
# Plot histogram with bins 
tobs_data = []
for row in temperature_data:
    tobs_data.append(row[0])

In [ ]:
plt.hist(tobs_data, bins=12, label='tobs', color = 'teal')
plt.xlabel("Temparture Observation Data")
plt.ylabel("Frequency")
plt.show()

# Temperature Analysis

In [ ]:
def temp_analysis(trip_start, trip_end):
    Start_Date = dt.strptime(trip_start, "%Y-%m-%d")
    Start_Date = Start_Date.replace(Start_Date.year - 1)
    End_Date = dt.strptime(trip_end, "%Y-%m-%d")
    End_Date = End_Date.replace(End_Date.year - 1)
    
    delta = End_Date - Start_Date
    date_range = []
    for i in range(delta.days + 1):
        date_range.append(Start_Date + timedelta(days=i))
       
    str_date_range = []
    for date in date_range:
        new_date = date.strftime("%Y-%m-%d")
        str_date_range.append(new_date)
           
    avg_temp = session.query(func.avg(Measurements.tobs))\
              .filter(Measurements.date.in_(str_date_range))[0][0]
    min_temp = session.query(func.min(Measurements.tobs))\
              .filter(Measurements.date.in_(str_date_range))[0][0]
    max_temp = session.query(func.max(Measurements.tobs))\
              .filter(Measurements.date.in_(str_date_range))[0][0]
       
    return avg_temp, min_temp, max_temp    

In [ ]:
# Prompt for trip dates to find temp data
trip_start = input("Please enter your trip's start date in yyyy-mm-dd format: ")
trip_end = input ("Please enter your trip's end date in yyyy-mm-dd format: ")

In [ ]:
trip_temp = temp_analysis(trip_start, trip_end)
print ("Here is the average, minimum and maximum temperature for your trip given the entered dates: " + str(Trip_Output))

In [ ]:
# Store info for the plot points
tavg = trip_temp[0]
tmin = trip_temp[1]
tmax = trip_temp[2]

In [ ]:
# Plot Bar chart 
plt.figure(figsize=(2,5))
plt.bar(1,height=tavg, yerr=(tmax-tmin), width=0.0001, alpha=0.5)
plt.title("Trip Avg Temperature")
plt.xticks([])
plt.ylim(0,100)
plt.ylabel("Temp (F)")
plt.show()